# Notebook 04: Momentum Indicators (RSI & MACD)

Welcome to Notebook 04! Learn to use RSI and MACD to identify overbought/oversold conditions.

## What You'll Learn

1. Relative Strength Index (RSI)
2. Identifying overbought and oversold conditions
3. RSI divergence patterns
4. MACD indicator components
5. MACD signal line crossovers
6. Combining RSI and MACD for stronger signals

## Time Required: 60-75 minutes

---

In [1]:
# Setup
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Try importing pandas_ta
try:
    import pandas_ta as ta
    print("✅ pandas-ta imported successfully!")
except ImportError:
    print("⚠️ pandas-ta not found. Install it with:")
    print("   pip install pandas-ta")
    print("\nYou need pandas-ta for RSI and MACD indicators.")

%matplotlib inline

# Try to use seaborn style, fall back to default if not available
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    try:
        plt.style.use('seaborn-darkgrid')
    except:
        plt.style.use('default')

# Fetch data
ticker = "0310.KL"  # UUE Holdings Berhad - Malaysian stock

try:
    stock = yf.Ticker(ticker)
    data = stock.history(period="6mo")
    
    if len(data) == 0:
        print(f"\n⚠️ No data returned for {ticker}")
        print("💡 Try: 1155.KL (Maybank) or 1295.KL (Public Bank)")
    else:
        # Calculate RSI
        data['RSI'] = ta.rsi(data['Close'], length=14)
        
        print(f"✅ Fetched data and calculated RSI for {ticker}")
        print(f"\nCurrent RSI: {data['RSI'].iloc[-1]:.2f}")
        
        if data['RSI'].iloc[-1] > 70:
            print("⚠️ Status: OVERBOUGHT - Consider selling or waiting")
        elif data['RSI'].iloc[-1] < 30:
            print("✅ Status: OVERSOLD - Potential buying opportunity")
        else:
            print("📊 Status: NEUTRAL - No extreme condition")
except ImportError:
    print("\n⚠️ pandas-ta is required for this notebook.")
    print("Install it with: pip install pandas-ta")
except Exception as e:
    print(f"\n⚠️ Error: {e}")
    print("Check your internet connection.")

✅ pandas-ta imported successfully!
✅ Fetched data and calculated RSI for 0310.KL

Current RSI: 39.02
📊 Status: NEUTRAL - No extreme condition


In [2]:
# Visualize RSI with price
if 'data' not in locals() or len(data) == 0:
    print("⚠️ Please run the previous cell first to fetch stock data.")
elif 'RSI' not in data.columns:
    print("⚠️ RSI not calculated. Please run the previous cell.")
else:
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        row_heights=[0.7, 0.3],
        subplot_titles=(f'{ticker} - UUE Holdings Berhad', 'RSI (14)')
    )

    # Price chart
    fig.add_trace(
        go.Scatter(x=data.index, y=data['Close'], name='Price', line=dict(color='blue')),
        row=1, col=1
    )

    # RSI chart
    fig.add_trace(
        go.Scatter(x=data.index, y=data['RSI'], name='RSI', line=dict(color='purple')),
        row=2, col=1
    )

    # Add overbought/oversold lines
    fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1, annotation_text="Overbought")
    fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1, annotation_text="Oversold")
    fig.add_hline(y=50, line_dash="dot", line_color="gray", row=2, col=1)

    fig.update_layout(height=800, template='plotly_white', showlegend=False, hovermode='x unified')
    fig.update_yaxes(title_text="Price (RM)", row=1, col=1)
    fig.update_yaxes(title_text="RSI", row=2, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)

    fig.show()

    print("✅ RSI chart created!")

✅ RSI chart created!


In [3]:
# Calculate MACD
if 'data' not in locals() or len(data) == 0:
    print("⚠️ Please run the previous cells first to fetch stock data.")
else:
    try:
        macd = ta.macd(data['Close'], fast=12, slow=26, signal=9)
        data['MACD'] = macd['MACD_12_26_9']
        data['MACD_signal'] = macd['MACDs_12_26_9']
        data['MACD_hist'] = macd['MACDh_12_26_9']

        print("✅ MACD calculated!")
        print(f"\nCurrent MACD: {data['MACD'].iloc[-1]:.4f}")
        print(f"Current Signal: {data['MACD_signal'].iloc[-1]:.4f}")
        print(f"Current Histogram: {data['MACD_hist'].iloc[-1]:.4f}")

        if data['MACD'].iloc[-1] > data['MACD_signal'].iloc[-1]:
            print("\n📈 Status: BULLISH - MACD above Signal")
        else:
            print("\n📉 Status: BEARISH - MACD below Signal")
    except ImportError:
        print("⚠️ pandas-ta is required. Install with: pip install pandas-ta")
    except Exception as e:
        print(f"⚠️ Error calculating MACD: {e}")

✅ MACD calculated!

Current MACD: -0.0092
Current Signal: -0.0034
Current Histogram: -0.0058

📉 Status: BEARISH - MACD below Signal


In [4]:
# Visualize MACD
if 'data' not in locals() or len(data) == 0:
    print("⚠️ Please run the previous cells first to fetch stock data.")
elif 'MACD' not in data.columns:
    print("⚠️ MACD not calculated. Please run the previous cell.")
else:
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        row_heights=[0.7, 0.3],
        subplot_titles=(f'{ticker} - UUE Holdings Berhad', 'MACD')
    )

    # Price
    fig.add_trace(
        go.Scatter(x=data.index, y=data['Close'], name='Price', line=dict(color='blue')),
        row=1, col=1
    )

    # MACD Line
    fig.add_trace(
        go.Scatter(x=data.index, y=data['MACD'], name='MACD', line=dict(color='blue')),
        row=2, col=1
    )

    # Signal Line
    fig.add_trace(
        go.Scatter(x=data.index, y=data['MACD_signal'], name='Signal', line=dict(color='orange')),
        row=2, col=1
    )

    # Histogram
    colors = ['green' if val >= 0 else 'red' for val in data['MACD_hist']]
    fig.add_trace(
        go.Bar(x=data.index, y=data['MACD_hist'], name='Histogram', marker_color=colors),
        row=2, col=1
    )

    fig.update_layout(height=800, template='plotly_white', hovermode='x unified')
    fig.update_yaxes(title_text="Price (RM)", row=1, col=1)
    fig.update_yaxes(title_text="MACD", row=2, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)

    fig.show()

    print("✅ MACD chart created!")

✅ MACD chart created!


In [5]:
# Identify confirmed signals
if 'data' not in locals() or len(data) == 0:
    print("⚠️ Please run the previous cells first to fetch stock data.")
elif 'RSI' not in data.columns or 'MACD' not in data.columns:
    print("⚠️ Please run previous cells to calculate RSI and MACD.")
else:
    # Strong buy: RSI < 40 AND MACD crosses above signal
    # Strong sell: RSI > 60 AND MACD crosses below signal

    data['MACD_above_signal'] = data['MACD'] > data['MACD_signal']
    data['MACD_cross'] = data['MACD_above_signal'].diff()

    # Find confirmed signals
    confirmed_buy = data[(data['MACD_cross'] == 1.0) & (data['RSI'] < 40)].copy()
    confirmed_sell = data[(data['MACD_cross'] == -1.0) & (data['RSI'] > 60)].copy()

    print("🎯 Confirmed Trading Signals\n")
    print("="*60)
    print(f"\nConfirmed BUY signals: {len(confirmed_buy)}")
    if len(confirmed_buy) > 0:
        print("\nRecent confirmed BUYs:")
        for date in confirmed_buy.tail(3).index:
            price = data.loc[date, 'Close']
            rsi = data.loc[date, 'RSI']
            print(f"   🟢 {date.strftime('%Y-%m-%d')} at RM {price:.2f} (RSI: {rsi:.1f})")

    print(f"\nConfirmed SELL signals: {len(confirmed_sell)}")
    if len(confirmed_sell) > 0:
        print("\nRecent confirmed SELLs:")
        for date in confirmed_sell.tail(3).index:
            price = data.loc[date, 'Close']
            rsi = data.loc[date, 'RSI']
            print(f"   🔴 {date.strftime('%Y-%m-%d')} at RM {price:.2f} (RSI: {rsi:.1f})")

🎯 Confirmed Trading Signals


Confirmed BUY signals: 0

Confirmed SELL signals: 0


## Part 3: Combining RSI and MACD

Using both indicators together provides **confirmation** and reduces false signals.

### Strong Buy Signal
- RSI crosses above 30 (oversold recovery)
- MACD crosses above Signal line
- Both indicators align

### Strong Sell Signal
- RSI crosses below 70 (overbought reversal)
- MACD crosses below Signal line
- Both indicators align

### Rule of Thumb
Wait for BOTH indicators to confirm before taking action!

## Key Takeaways

### ✅ What You Learned

1. **RSI** identifies overbought/oversold conditions
2. **MACD** shows momentum and trend changes
3. **Combining indicators** provides stronger signals
4. Always wait for **confirmation** before trading

### 📚 Next Steps

Notebook 05: Volume Analysis

---

**Happy Learning! 📚📊**